# 📊 SYNTHÈSE COMPLÈTE DES ANALYSES TEMPORELLES


**Date**: 17 octobre 2025  
**Dataset**: MangeTaMain - Analyses Rating Temporelles  
**Status**: ✅ Données nettoyées (ratings=0 supprimés)

# 🎯 SYNTHÈSE COMPLÈTE DES ANALYSES TEMPORELLES

**Exécution complète des 3 analyses prioritaires avec résultats business**

## 📋 Analyses implémentées:
1. **🎯 Weekend Analysis** - Patterns weekend vs semaine
2. **🌸 Seasonality Analysis** - Cycles saisonniers et mensuels  
3. **📈 Long Term Analysis** - Tendances et évolutions temporelles

---

In [ ]:
# 📊 SYNTHÈSE COMPLÈTE - ANALYSES MANGETAMAIN
print("=" * 80)
print("🚀 SYNTHÈSE COMPLÈTE DES ANALYSES - MANGETAMAIN KITCHEN")
print("=" * 80)

# Imports complets
import sys
from pathlib import Path
sys.path.append(str(Path(".").resolve()))

import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import spearmanr, mannwhitneyu, linregress
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuration visuelle
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Imports réussis")

# Chargement des utilities
exec(open("00_data_utils.ipynb").read())

# Test des données
df_clean = load_clean_interactions()
print(f"✅ Données chargées: {df_clean.shape}")
print(f"📊 Colonnes disponibles: {list(df_clean.columns)}")

# Vérification qualité
quality_report = check_data_quality(df_clean)
print("\n🔍 RAPPORT QUALITÉ:")
for metric, value in quality_report.items():
    print(f"  • {metric}: {value}")

print(f"\n🎯 Prêt pour les 3 analyses:")
print(f"  1. 📅 Weekend Effect Analysis")
print(f"  2. 🌍 Seasonality Analysis")
print(f"  3. 📈 Long Term Trends Analysis")

# Préparation des datasets globaux
monthly_stats = df_clean.group_by(["year", "month"]).agg([
    pl.col("rating").mean().alias("mean_rating"),
    pl.col("rating").median().alias("median_rating"),
    pl.col("rating").std().alias("std_rating"),
    pl.len().alias("n_interactions")
]).sort(["year", "month"])

daily_stats = df_clean.group_by(["year", "month", "day"]).agg([
    pl.col("rating").mean().alias("mean_rating"),
    pl.len().alias("n_interactions")
]).sort(["year", "month", "day"])

weekend_stats = df_clean.group_by(["is_weekend"]).agg([
    pl.col("rating").mean().alias("mean_rating"),
    pl.col("rating").median().alias("median_rating"),
    pl.col("rating").std().alias("std_rating"),
    pl.len().alias("n_interactions")
])

seasonal_stats = df_clean.group_by(["season"]).agg([
    pl.col("rating").mean().alias("mean_rating"),
    pl.col("rating").median().alias("median_rating"),
    pl.col("rating").std().alias("std_rating"),
    pl.len().alias("n_interactions")
])

print(f"\n? Datasets préparés:")
print(f"  • Monthly: {monthly_stats.shape}")
print(f"  • Daily: {daily_stats.shape}")
print(f"  • Weekend: {weekend_stats.shape}")
print(f"  • Seasonal: {seasonal_stats.shape}")

print("\n✅ INITIALISATION TERMINÉE - Prêt pour les analyses!")

## 🎯 1. WEEKEND ANALYSIS - Résultats Complets

In [ ]:
print("🎯 WEEKEND ANALYSIS")
print("=" * 30)

# 1.1 Weekend vs Weekday Test
weekend_comparison = df_clean.group_by("is_weekend").agg([
    pl.col("rating").mean().alias("mean_rating"),
    pl.col("rating").std().alias("std_rating"),
    pl.len().alias("n_interactions")
]).sort("is_weekend").to_pandas()

weekend_ratings = df_clean.filter(pl.col("is_weekend") == 1)["rating"].to_numpy()
weekday_ratings = df_clean.filter(pl.col("is_weekend") == 0)["rating"].to_numpy()

# Test statistique
statistic, p_value = mannwhitneyu(weekend_ratings, weekday_ratings, alternative='two-sided')
pooled_std = np.sqrt((np.std(weekend_ratings, ddof=1)**2 + np.std(weekday_ratings, ddof=1)**2) / 2)
cohens_d = (np.mean(weekend_ratings) - np.mean(weekday_ratings)) / pooled_std

print(f"\n📊 Weekend vs Semaine:")
print(f"• Weekend: {np.mean(weekend_ratings):.4f} ± {np.std(weekend_ratings):.4f}")
print(f"• Semaine: {np.mean(weekday_ratings):.4f} ± {np.std(weekday_ratings):.4f}")
print(f"• Différence: {np.mean(weekend_ratings) - np.mean(weekday_ratings):+.4f} points")
print(f"• P-valeur: {p_value:.4f} ({'SIGNIFICATIF' if p_value < 0.05 else 'NON SIGNIFICATIF'})")
print(f"• Cohen's d: {cohens_d:.4f} ({'Petit' if abs(cohens_d) < 0.2 else 'Moyen' if abs(cohens_d) < 0.5 else 'Grand'} effet)")

# 1.2 Patterns par jour
day_names = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]
weekday_patterns = df_clean.group_by("weekday").agg([
    pl.col("rating").mean().alias("mean_rating"),
    pl.len().alias("n_interactions")
]).sort("weekday").to_pandas()
weekday_patterns["day_name"] = day_names

best_day = weekday_patterns.loc[weekday_patterns["mean_rating"].idxmax()]
worst_day = weekday_patterns.loc[weekday_patterns["mean_rating"].idxmin()]

print(f"\n📅 Patterns journaliers:")
print(f"• Meilleur jour: {best_day['day_name']} ({best_day['mean_rating']:.4f})")
print(f"• Pire jour: {worst_day['day_name']} ({worst_day['mean_rating']:.4f})")
print(f"• Amplitude: {weekday_patterns['mean_rating'].max() - weekday_patterns['mean_rating'].min():.4f} points")

# Visualisation Weekend
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=["Weekend vs Semaine", "Patterns par jour"],
    specs=[[{"type": "xy"}, {"type": "xy"}]]
)

# Violin plot weekend
fig.add_trace(go.Violin(
    y=weekday_ratings,
    name="Semaine",
    box_visible=True,
    fillcolor="lightblue"
), row=1, col=1)

fig.add_trace(go.Violin(
    y=weekend_ratings,
    name="Weekend",
    box_visible=True,
    fillcolor="lightcoral"
), row=1, col=1)

# Bar chart journalier
fig.add_trace(go.Bar(
    x=weekday_patterns["day_name"],
    y=weekday_patterns["mean_rating"],
    text=[f"{val:.3f}" for val in weekday_patterns["mean_rating"]],
    textposition="outside",
    marker_color=["red" if i in [0,1] else "orange" if i in [2,3] else "green" for i in range(7)]
), row=1, col=2)

fig.update_layout(
    title=f"Weekend Analysis (Cohen's d = {cohens_d:.3f}, p = {p_value:.4f})",
    height=500,
    showlegend=False
)

fig.show()

weekend_result = {
    "effet_weekend": np.mean(weekend_ratings) - np.mean(weekday_ratings),
    "significatif": p_value < 0.05,
    "meilleur_jour": best_day['day_name'],
    "pire_jour": worst_day['day_name']
}

print(f"✅ Weekend Analysis terminée!")

## 🌸 2. SEASONALITY ANALYSIS - Résultats Complets

In [ ]:
print("🌸 SEASONALITY ANALYSIS")
print("=" * 30)

# 2.1 Analyse saisonnière
season_order = ["Spring", "Summer", "Autumn", "Winter"]
seasonal_ratings = df_clean.group_by("season").agg([
    pl.col("rating").mean().alias("mean_rating"),
    pl.col("rating").std().alias("std_rating"),
    pl.len().alias("n_interactions")
]).to_pandas()

seasonal_ratings = seasonal_ratings.set_index('season').loc[season_order].reset_index()

# Test ANOVA
season_groups = [df_clean.filter(pl.col("season") == season)["rating"].to_list() 
                for season in season_order]
f_stat, p_anova = f_oneway(*season_groups)

best_season = seasonal_ratings.loc[seasonal_ratings['mean_rating'].idxmax(), 'season']
worst_season = seasonal_ratings.loc[seasonal_ratings['mean_rating'].idxmin(), 'season']
seasonal_range = seasonal_ratings['mean_rating'].max() - seasonal_ratings['mean_rating'].min()

print(f"\n🌸 Variations saisonnières:")
display(seasonal_ratings[['season', 'mean_rating', 'std_rating', 'n_interactions']])
print(f"• ANOVA p-valeur: {p_anova:.4f} ({'SIGNIFICATIF' if p_anova < 0.05 else 'NON SIGNIFICATIF'})")
print(f"• Meilleure saison: {best_season} ({seasonal_ratings.loc[seasonal_ratings['season']==best_season, 'mean_rating'].iloc[0]:.4f})")
print(f"• Pire saison: {worst_season} ({seasonal_ratings.loc[seasonal_ratings['season']==worst_season, 'mean_rating'].iloc[0]:.4f})")
print(f"• Écart saisonnier: {seasonal_range:.4f} points")

# 2.2 Patterns mensuels
monthly_patterns = df_clean.group_by("month").agg([
    pl.col("rating").mean().alias("mean_rating"),
    pl.len().alias("n_interactions")
]).sort("month").to_pandas()

month_names = ["Jan", "Fév", "Mar", "Avr", "Mai", "Jun", "Jul", "Aoû", "Sep", "Oct", "Nov", "Déc"]
monthly_patterns["month_name"] = month_names

best_month = monthly_patterns.loc[monthly_patterns['mean_rating'].idxmax()]
worst_month = monthly_patterns.loc[monthly_patterns['mean_rating'].idxmin()]

print(f"\n📅 Patterns mensuels:")
print(f"• Meilleur mois: {best_month['month_name']} ({best_month['mean_rating']:.4f})")
print(f"• Pire mois: {worst_month['month_name']} ({worst_month['mean_rating']:.4f})")

# Visualisation Seasonality
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["Ratings par saison", "Pattern polaire mensuel", "Heatmap année×mois", "Cyclicité FFT"],
    specs=[[{"type": "xy"}, {"type": "polar"}], 
           [{"type": "xy"}, {"type": "xy"}]]
)

# 1. Box plots saisons
season_colors = {"Spring": "lightgreen", "Summer": "gold", "Autumn": "orange", "Winter": "lightblue"}
for season in season_order:
    season_data = df_clean.filter(pl.col("season") == season)["rating"].to_list()
    fig.add_trace(go.Box(
        y=season_data,
        name=season,
        marker_color=season_colors[season]
    ), row=1, col=1)

# 2. Radar mensuel
fig.add_trace(go.Scatterpolar(
    r=monthly_patterns["mean_rating"],
    theta=monthly_patterns["month_name"],
    fill='toself',
    name='Rating mensuel'
), row=1, col=2)

# 3. Heatmap (simplifié)
year_month_grid = df_clean.group_by(["year", "month"]).agg(
    pl.col("rating").mean().alias("mean_rating")
).to_pandas()
heatmap_data = year_month_grid.pivot(index='year', columns='month', values='mean_rating')

fig.add_trace(go.Heatmap(
    z=heatmap_data.values,
    x=[month_names[i-1] for i in heatmap_data.columns],
    y=heatmap_data.index,
    colorscale="RdYlBu_r"
), row=2, col=1)

# 4. FFT analysis
monthly_series = monthly_patterns["mean_rating"].values
fft_values = np.abs(fft(monthly_series))
frequencies = fftfreq(len(monthly_series))

fig.add_trace(go.Scatter(
    x=frequencies[:len(frequencies)//2],
    y=fft_values[:len(fft_values)//2],
    mode='lines+markers',
    name='Spectrum'
), row=2, col=2)

fig.update_layout(
    title=f"Seasonality Analysis (ANOVA p = {p_anova:.4f})",
    height=800,
    showlegend=False
)

fig.show()

seasonality_result = {
    "ecart_saisonnier": seasonal_range,
    "significatif": p_anova < 0.05,
    "meilleure_saison": best_season,
    "pire_saison": worst_season,
    "meilleur_mois": best_month['month_name'],
    "pire_mois": worst_month['month_name']
}

print(f"✅ Seasonality Analysis terminée!")

## 📈 3. LONG TERM ANALYSIS - Résultats Complets

In [ ]:
# ? ANALYSE 3: LONG TERM TRENDS
print("=" * 60)
print("📈 ANALYSE 3: TENDANCES LONG TERME")
print("=" * 60)

def mann_kendall_test(data):
    """Test de Mann-Kendall pour détecter les tendances monotones"""
    n = len(data)
    s = 0
    
    for i in range(n-1):
        for j in range(i+1, n):
            if data[j] > data[i]:
                s += 1
            elif data[j] < data[i]:
                s -= 1
    
    var_s = n * (n-1) * (2*n+5) / 18
    
    if s > 0:
        z = (s - 1) / np.sqrt(var_s)
    elif s < 0:
        z = (s + 1) / np.sqrt(var_s)
    else:
        z = 0
    
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    return z, p_value, s

# 1. Test de tendance sur les ratings mensuels
monthly_analysis = monthly_stats.to_pandas()
monthly_ratings = monthly_analysis['mean_rating'].values

z_mk, p_mk, s_mk = mann_kendall_test(monthly_ratings)

print(f"🔍 Test de Mann-Kendall - Tendance Long Terme:")
print(f"Statistique S: {s_mk}")
print(f"Z-score: {z_mk:.4f}")
print(f"P-value: {p_mk:.4f}")

if p_mk < 0.05:
    direction = "HAUSSE" if s_mk > 0 else "BAISSE"
    print(f"✅ TENDANCE SIGNIFICATIVE: {direction}")
else:
    print(f"❌ Pas de tendance claire détectée")

# 2. Régression pour quantifier l'évolution
X = np.arange(len(monthly_analysis))
y = monthly_analysis['mean_rating'].values
slope, intercept, r_value, p_value, std_err = linregress(X, y)

print(f"\n? Régression linéaire:")
print(f"Pente: {slope:.6f} points/mois")
print(f"R²: {r_value**2:.4f}")
print(f"Evolution annuelle: {slope * 12:.4f} points/an")

# 3. Détection des points de rupture (changements structurels)
def detect_breakpoints(data, window=6):
    """Détection simple de points de rupture par variance glissante"""
    rolling_mean = pd.Series(data).rolling(window).mean()
    rolling_std = pd.Series(data).rolling(window).std()
    
    # Normalisation pour détecter les anomalies
    normalized = (pd.Series(data) - rolling_mean) / rolling_std
    breakpoints = np.where(np.abs(normalized) > 2)[0]  # > 2 sigma
    
    return breakpoints

breakpoints = detect_breakpoints(monthly_ratings)
print(f"\n🔍 Points de rupture détectés: {len(breakpoints)}")
if len(breakpoints) > 0:
    print(f"Mois concernés: {breakpoints}")

# 4. Evolution des power users
power_users_evolution = df_clean.filter(
    pl.col("user_id").is_in(
        df_clean.group_by("user_id").len().filter(
            pl.col("len") >= pl.col("len").quantile(0.8)
        ).select("user_id").to_series()
    )
).group_by(["year", "month"]).agg([
    pl.col("rating").mean().alias("power_user_avg_rating"),
    pl.len().alias("power_user_interactions")
]).sort(["year", "month"]).to_pandas()

# Corrélation entre activité générale et power users
if len(power_users_evolution) > 5:
    corr_power, p_power = spearmanr(
        monthly_analysis['mean_rating'].values[:len(power_users_evolution)],
        power_users_evolution['power_user_avg_rating'].values
    )
    print(f"\n👥 Power Users - Corrélation avec tendance générale:")
    print(f"ρ = {corr_power:.3f} (p = {p_power:.4f})")

# 5. Visualisation complète
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        "Evolution Ratings Temporelle",
        "Volume vs Qualité",
        "Power Users vs Général",
        "Distribution par Période"
    ]
)

# Graphique 1: Evolution temporelle avec tendance
dates = pd.to_datetime(monthly_analysis[['year', 'month']].assign(day=1))
fig.add_trace(go.Scatter(
    x=dates,
    y=monthly_analysis['mean_rating'],
    mode='lines+markers',
    name='Rating moyen',
    line=dict(color='blue', width=2)
), row=1, col=1)

# Ligne de tendance
trend_line = intercept + slope * X
fig.add_trace(go.Scatter(
    x=dates,
    y=trend_line,
    mode='lines',
    name=f'Tendance ({slope:.4f}/mois)',
    line=dict(color='red', dash='dash', width=2)
), row=1, col=1)

# Marquage des breakpoints
if len(breakpoints) > 0:
    for bp in breakpoints:
        if bp < len(dates):
            fig.add_vline(
                x=dates.iloc[bp],
                line_dash="dot",
                line_color="orange",
                row=1, col=1
            )

# Graphique 2: Volume vs Qualité
fig.add_trace(go.Scatter(
    x=monthly_analysis['n_interactions'],
    y=monthly_analysis['mean_rating'],
    mode='markers',
    name='Volume-Qualité',
    marker=dict(
        size=8,
        color=monthly_analysis.index,
        colorscale='Viridis',
        showscale=True
    )
), row=1, col=2)

# Graphique 3: Power users comparison (si données disponibles)
if len(power_users_evolution) > 0:
    dates_power = pd.to_datetime(power_users_evolution[['year', 'month']].assign(day=1))
    fig.add_trace(go.Scatter(
        x=dates_power,
        y=power_users_evolution['power_user_avg_rating'],
        mode='lines',
        name='Power Users',
        line=dict(color='green')
    ), row=2, col=1)
    
    fig.add_trace(go.Scatter(
        x=dates[:len(power_users_evolution)],
        y=monthly_analysis['mean_rating'].values[:len(power_users_evolution)],
        mode='lines',
        name='Utilisateurs généraux',
        line=dict(color='blue')
    ), row=2, col=1)

# Graphique 4: Distribution par période
periods = ['Début', 'Milieu', 'Récent']
n_total = len(monthly_ratings)
period_size = n_total // 3

period_means = [
    np.mean(monthly_ratings[:period_size]),
    np.mean(monthly_ratings[period_size:2*period_size]),
    np.mean(monthly_ratings[2*period_size:])
]

fig.add_trace(go.Bar(
    x=periods,
    y=period_means,
    name='Rating par période',
    marker_color=['lightblue', 'lightgreen', 'lightcoral']
), row=2, col=2)

fig.update_layout(
    title=f"Long Term Analysis - Tendance: {slope:+.4f}/mois (p={p_mk:.4f})",
    height=800,
    showlegend=True
)

fig.show()

# 6. Synthèse Business
print(f"\n" + "="*50)
print(f"💼 SYNTHÈSE BUSINESS - LONG TERME")
print(f"="*50)

if p_mk < 0.05:
    trend_desc = "amélioration" if slope > 0 else "dégradation"
    annual_change = slope * 12
    print(f"📈 TENDANCE CONFIRMÉE: {trend_desc} de {abs(annual_change):.3f} points/an")
    print(f"🎯 En {int(1/abs(slope))} mois: changement d'1 point de rating")
else:
    print(f"📊 STABILITÉ: Pas d'évolution significative détectée")

print(f"🔍 Fiabilité du modèle: R² = {r_value**2:.3f}")

if len(breakpoints) > 0:
    print(f"⚠️  RUPTURES DÉTECTÉES: {len(breakpoints)} changements structurels")
else:
    print(f"✅ ÉVOLUTION RÉGULIÈRE: Pas de rupture majeure")

# Recommandations
print(f"\n📋 RECOMMANDATIONS:")
if slope > 0 and p_mk < 0.05:
    print(f"  • ✅ Maintenir la stratégie actuelle (amélioration continue)")
    print(f"  • 🎯 Identifier les facteurs de cette amélioration")
elif slope < 0 and p_mk < 0.05:
    print(f"  • ⚠️  Investiguer les causes de dégradation")
    print(f"  • 🔧 Plan d'action correctif nécessaire")
else:
    print(f"  • 📊 Surveillance continue requise")
    print(f"  • 💡 Opportunité d'innovation pour créer une tendance positive")

print(f"\n✅ ANALYSE LONG TERME TERMINÉE")

## 🎯 4. SYNTHÈSE FINALE & IMPLICATIONS BUSINESS

In [ ]:
print("🎯 SYNTHÈSE FINALE DES 3 ANALYSES")
print("=" * 50)

# Compilation des résultats
results_summary = {
    "🎯 WEEKEND ANALYSIS": {
        "Effet weekend": f"{weekend_result['effet_weekend']:+.4f} points",
        "Significatif": "OUI" if weekend_result['significatif'] else "NON",
        "Meilleur jour": weekend_result['meilleur_jour'],
        "Pire jour": weekend_result['pire_jour'],
        "Implication": "Weekend plus généreux" if weekend_result['effet_weekend'] > 0 else "Weekend plus sévère"
    },
    
    "🌸 SEASONALITY ANALYSIS": {
        "Écart saisonnier": f"{seasonality_result['ecart_saisonnier']:.4f} points",
        "Significatif": "OUI" if seasonality_result['significatif'] else "NON",
        "Meilleure saison": seasonality_result['meilleure_saison'],
        "Pire saison": seasonality_result['pire_saison'],
        "Meilleur mois": seasonality_result['meilleur_mois'],
        "Pire mois": seasonality_result['pire_mois']
    },
    
    "📈 LONG TERM ANALYSIS": {
        "Tendance mensuelle": f"{longterm_result['tendance']:+.6f} points/mois",
        "Significatif": "OUI" if longterm_result['significatif'] else "NON",
        "Évolution annuelle": f"{longterm_result['evolution_annuelle']:+.4f} points/an",
        "Volume-Qualité": f"{longterm_result['correlation_volume_qualite']:+.4f}",
        "Direction": "Amélioration" if longterm_result['tendance'] > 0 else "Dégradation"
    }
}

for analysis, metrics in results_summary.items():
    print(f"\n{analysis}:")
    for metric, value in metrics.items():
        print(f"  • {metric}: {value}")

print("\n" + "=" * 50)
print("💡 IMPLICATIONS BUSINESS PRIORITAIRES")
print("=" * 50)

implications = [
    "🎯 TIMING OPTIMAL:",
    f"  • Lancer produits le {weekend_result['meilleur_jour'].lower()}",
    f"  • Éviter le {weekend_result['pire_jour'].lower()}",
    f"  • Saison optimale: {seasonality_result['meilleure_saison']}",
    f"  • Éviter: {seasonality_result['pire_saison']} et {seasonality_result['pire_mois']}",
    "",
    "📈 STRATÉGIE TEMPORELLE:",
    f"  • Tendance: {'Positive' if longterm_result['tendance'] > 0 else 'Négative'} ({longterm_result['evolution_annuelle']:+.4f}/an)",
    f"  • Volume-Qualité: {'Dilution détectée' if longterm_result['correlation_volume_qualite'] < -0.3 else 'Pas de dilution'}",
    "",
    "🚀 ACTIONS IMMÉDIATES:",
    "  1. Implémenter calendrier éditorial basé sur patterns découverts",
    "  2. Optimiser timing des lancements produits",
    "  3. Monitoring continue des tendances détectées",
    "  4. Segmentation communications temporelles",
    "",
    "🎯 MÉTRIQUES À SURVEILLER:",
    "  • Évolution différentiel weekend-semaine",
    "  • Stabilité patterns saisonniers", 
    "  • Dérive tendance long terme",
    "  • Corrélation volume-qualité"
]

for implication in implications:
    print(implication)

# Calcul ROI potentiel
print("\n💰 ROI ESTIMÉ:")
effect_size = abs(weekend_result['effet_weekend']) + seasonality_result['ecart_saisonnier']
optimization_potential = effect_size * 0.5  # 50% d'optimisation réaliste

print(f"  • Effet total détectable: {effect_size:.4f} points")
print(f"  • Potentiel d'optimisation: {optimization_potential:.4f} points")
print(f"  • Impact relatif: {optimization_potential/df_clean['rating'].mean()*100:.2f}% d'amélioration possible")

print("\n✅ SYNTHÈSE COMPLÈTE TERMINÉE !")
print("🚀 Prêt pour implémentation business !")